FRP

In [ ]:
# Turn point dataset into gridded dataset, where its horizontal resolution is 0.75º*0.75º

import pandas as pd
import numpy as np
import xarray as xr
import os

# === INPUT CSV FILE ===
csv_path = r"E:\IPMA\FRP\FRP_2000_2024.csv"

# === EXTRACT DIRECTORY AND CREATE OUTPUT FILENAME ===
base_dir = os.path.dirname(csv_path)
output_filename = "FRP_2000_2024.nc"
output_path = os.path.join(base_dir, output_filename)

# === GRID PARAMETERS ===
lat_res = 0.75
lon_res = 0.75
lat_min, lat_max = 34, 66
lon_min, lon_max = -12, 36

lat_bins = np.arange(lat_min, lat_max + lat_res, lat_res)
lon_bins = np.arange(lon_min, lon_max + lon_res, lon_res)
lat_centers = lat_bins[:-1]
lon_centers = lon_bins[:-1]

# === LOAD AND PREPARE DATA ===
print("Loading and filtering data...")
df = pd.read_csv(csv_path)
df = df[(df['latitude'] >= lat_min) & (df['latitude'] <= lat_max) &
        (df['longitude'] >= lon_min) & (df['longitude'] <= lon_max)]

# Ensure date is datetime
df['acq_date'] = pd.to_datetime(df['acq_date'])

# Assign grid cell
print("Assigning points to grid cells...")
df['lat_bin'] = pd.cut(df['latitude'], bins=lat_bins, labels=lat_centers)
df['lon_bin'] = pd.cut(df['longitude'], bins=lon_bins, labels=lon_centers)
df.dropna(subset=['lat_bin', 'lon_bin'], inplace=True)

# Convert bins to float
df['lat_bin'] = df['lat_bin'].astype(float)
df['lon_bin'] = df['lon_bin'].astype(float)

# === AGGREGATE: DAILY SUM AND COUNT ===
print("Aggregating daily FRP and count...")
grouped = df.groupby(['acq_date', 'lat_bin', 'lon_bin'])

# Create MultiIndexed DataFrames
frp_sum = grouped['frp'].sum().reset_index(name='FRP')
count = grouped.size().reset_index(name='count')

# Merge both
merged = pd.merge(frp_sum, count, on=['acq_date', 'lat_bin', 'lon_bin'])

# Create full 3D arrays
print("Creating 3D arrays...")
all_dates = pd.date_range(df['acq_date'].min(), df['acq_date'].max(), freq='D')
frp_array = np.zeros((len(all_dates), len(lat_centers), len(lon_centers)))
count_array = np.zeros_like(frp_array)

lat_to_idx = {lat: i for i, lat in enumerate(lat_centers)}
lon_to_idx = {lon: i for i, lon in enumerate(lon_centers)}
date_to_idx = {date: i for i, date in enumerate(all_dates)}

for _, row in merged.iterrows():
    d, lat, lon = row['acq_date'], row['lat_bin'], row['lon_bin']
    i, j, k = date_to_idx[d], lat_to_idx[lat], lon_to_idx[lon]
    frp_array[i, j, k] = row['FRP']
    count_array[i, j, k] = row['count']

# === CREATE xarray.Dataset ===
print("Building xarray dataset...")
ds = xr.Dataset(
    {
        "FRP": (["time", "latitude", "longitude"], frp_array),
        "count": (["time", "latitude", "longitude"], count_array),
    },
    coords={
        "time": all_dates,
        "latitude": lat_centers,
        "longitude": lon_centers,
    },
    attrs={
        "title": "Daily Gridded FRP Dataset (0.75° × 0.75°)",
        "source_file": os.path.basename(csv_path),
        "grid_resolution": "0.75 degrees",
        "description": "Includes daily total FRP and count of detections per grid cell"
    }
)

# === SAVE TO NETCDF ===
print(f"Saving to NetCDF: {output_path}")
ds.to_netcdf(output_path)
print("Done.")

Loading and filtering data...
Assigning points to grid cells...
Aggregating daily FRP and count...
Creating 3D arrays...
Building xarray dataset...
Saving to NetCDF: E:\IPMA\FRP\FRP_2000_2024.nc
Done.


In [ ]:
#Time series study area, iberia, greece, portugal, south europe with daily averages

import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates

# Load the NetCDF file
file_path = r"E:\IPMA\FRP\daily_frp_stats.nc"
ds = xr.open_dataset(file_path)

# Define regions: (lat_min, lat_max, lon_min, lon_max)
regions = {
    "Study Area": (34, 66, -12, 36),
    "Iberia": (35, 45, -10, 5),
    "Portugal": (36, 43, -10, -6),
    "Greece": (34, 42, 19, 29),
    "South Europe": (34, 45, -10, 29),
}

# Stack Year, Month, Day into a single dimension
co = ds['Mean'].stack(time=('Year', 'Month', 'Day'))

# Build datetime index from stacked coordinates
year = co['Year'].values
month = co['Month'].values
day = co['Day'].values

# Create datetime objects, filter out invalid ones
dates = []
valid_indices = []

for i in range(len(year)):
    try:
        dt = pd.Timestamp(year=int(year[i]), month=int(month[i]), day=int(day[i]))
        dates.append(dt)
        valid_indices.append(i)
    except ValueError:
        # Skip invalid dates
        continue

# Select only valid entries
co = co.isel(time=valid_indices)

# Dictionary to store time series for each region
region_series = {}

for name, (lat_min, lat_max, lon_min, lon_max) in regions.items():
    subset = co.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max))
    regional_mean = subset.mean(dim=['latitude', 'longitude'])
    region_series[name] = pd.Series(regional_mean.values, index=pd.DatetimeIndex(dates))

# Plotting
plt.figure(figsize=(14, 7))

for name, series in region_series.items():
    plt.plot(series.index, series, label=name)

plt.title('Daily Mean FRP by Region')
plt.xlabel('Year')
plt.ylabel('FRP')
plt.grid(True)
plt.legend()

# Format x-axis to show every year
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.YearLocator(1))  # One tick per year
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=45)  # Optional: tilt labels for readability

plt.tight_layout()
plt.show()

In [ ]:
#Variability boxplot

In [ ]:
#Excedencias tabela total

In [ ]:
#Fire days

In [ ]:
#No Fire days

In [ ]:
#FRP para grelha